In [1]:
# Yuliya Akchurina 
# CISC6210 Natural Language Processing
# Fall 2021
# Homework_1

In [2]:
# Part 1. 
# Read poems from .txt files in local folder into a table which has five columns - Author, Title, Tags, Body, Link. 
# Each poem is a row of the table.
# Save the table into a local Microsoft Excel Worksheet named CleanOutputLoveOutput.xlsx

In [3]:
import nltk
import pandas as pd
import os
import glob
import re 
from nltk.tokenize import word_tokenize
from gensim.parsing.preprocessing import remove_stopwords
from nltk.corpus import stopwords
stop = stopwords.words('english')

from nltk.stem import PorterStemmer

In [4]:
# Folder Path
folder_path = "C:\\Users\\Yuliya.DESKTOP-FMS37R3\\Documents\\Fordham Fall 2021 Classes\\CISC6210 NLP\\LoveOutput\\"

# Change the directory
os.chdir(folder_path)

# get file names into a list
filenames = [i for i in glob.glob("*.txt")]
#print(filenames)

In [5]:
#check how many files we have
print(len(filenames))

2232


In [6]:
#Load all poems to create list of dataframes

df = [pd.read_csv(file, sep = "\n\n", header=None) for file in filenames]

C:\Users\Yuliya.DESKTOP-FMS37R3\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until


In [7]:
#check size
print(len(df))

2232


In [8]:
#print(df[0:2])

In [9]:
#transpose each dataframe in the list and concatenate the transposed dataframes into a single dataframe
for i in range(0,len(df)):
    df[i] = df[i].T
    
poem_data = pd.concat(df)                   
poem_data = poem_data.reset_index(drop=True)

In [10]:
#rename columns
poem_data = poem_data.rename(columns={0: "Title_Author", 1: "Tags", 2: "Body", 3: "Link"})

In [11]:
#confirm the columns were renamed
print(poem_data[0:2])

              Title_Author                                               Tags  \
0    1979 By Roddy Lumsden  Living ;Life Choices ;Time & Brevity ;Love ;De...   
1  66 By Suzanne Gardinier            Living ;Love ;Relationships ;Free Verse   

                                                Body  \
0  They arrived at the desk of the Hotel Duncan<b...   
1  I'm used to the emperor's bitterness<br> I can...   

                                                Link  
0  original link: https://www.poetryfoundation.or...  
1  original link: https://www.poetryfoundation.or...  


In [12]:
# remove "original link: " from Link column
poem_data['Link'] = poem_data['Link'].str.replace('original link: ', '', regex=True)


In [13]:
#strip whitespace 
#poem_data['Link'] = poem_data['Link'].str.strip()
#poem_data['Link'] = poem_data['Link'].str.rstrip()  #remove trailing space
#poem_data['Link'] = poem_data['Link'].str.replace(" ", "") 

In [14]:
#confrim
#print(poem_data[0:2])

In [15]:
# split title and author into two columns
poem_data["Title_Author"] = poem_data["Title_Author"].str.replace(' By ', '_', regex=True)
poem_data[["Title", "Author"]] = poem_data["Title_Author"].str.rsplit("_", 1, expand=True)


In [16]:
#rearrange columns in dataframe 
poem_data = poem_data[['Author', 'Title', 'Tags','Body','Link','Title_Author']]

# Drop last column of a dataframe
poem_data = poem_data.iloc[: , :-1]

In [17]:
# replace with line breaks replaced by [L] and paragraph breaks replaced by [P]

poem_data['Body'] = poem_data['Body'].str.replace('<br>', '[L]', regex=False)
poem_data['Body'] = poem_data['Body'].str.replace('<br><br>', '[P]', regex=False)
poem_data['Body'] = poem_data['Body'].str.replace('<p>', '[P]', regex=False)
poem_data['Body'] = poem_data['Body'].str.replace('</p>', '[P]', regex=False)

In [18]:
#strip remaining HTML tags 

poem_data['Body'] = poem_data['Body'].str.replace(r'<[^<>]*>', '', regex=True)
poem_data['Title'] = poem_data['Title'].str.replace(r'<[^<>]*>', '', regex=True)

poem_data['Author'] = poem_data['Author'].str.replace(r'[&#039;]', '', regex=True)
poem_data['Title'] = poem_data['Title'].str.replace(r'[&#039;]', '', regex=True)


In [21]:
# identify missing values in the dataframe
# if the missing values anything but a link remove rows containing missing values
poem_data.isnull()

,Author,Title,Tags,Body,Link
0,False,False,False,False,False
1,False,False,False,False,False
2,False,False,False,False,False
3,False,False,False,False,False
4,False,False,False,False,False
...,...,...,...,...,...
2227,False,False,False,False,False
2228,False,False,False,False,False
2229,False,False,False,False,False
2230,False,False,False,False,False


In [22]:
poem_data.describe()

,Author,Title,Tags,Body,Link
count,2232,2232,2232,2040,2040
unique,1041,2183,1860,2037,2040
top,William Shakespeare,Song,Love ;Realistic & Complicated,By ...,https://www.poetryfoundation.org/poems/50338/w...
freq,52,11,35,3,1


In [23]:
#verify how many values are missing in each column
poem_data.isnull().sum()

Author      0
Title       0
Tags        0
Body      192
Link      192
dtype: int64

In [24]:
# drop rows if values are missing in any column but "Link" since link is optional 
poem_data = poem_data.dropna(axis=0, how='any', subset=['Author', 'Title', 'Tags', 'Body'])

In [25]:
# check new size (2040, 5)
poem_data.shape

(2040, 5)

In [26]:
#verify how many values are missing in each column left
poem_data.isnull().sum()

Author    0
Title     0
Tags      0
Body      0
Link      0
dtype: int64

In [27]:
#save locally to excel. 
poem_data.to_excel(r"C:\Users\Yuliya.DESKTOP-FMS37R3\Documents\Fordham Fall 2021 Classes\CISC6210 NLP\Homework\CleanOutputLoveOutput.xlsx",
                index=False, header=True)


In [28]:
# Once you processed all poems, show information of the table on screen, such as total number of poems stored, 
# total number of authors, sort authors by the amount of their poems collected in the table, 
# and show top 20 authors.

In [29]:
num_poem = len(poem_data["Title"])
num_author = len(poem_data["Author"])

# get top 20 most frequent authors
n = 20
top20 = poem_data['Author'].value_counts()[:n].index.tolist()


In [30]:
#poem_data['Author'].value_counts()

In [31]:
print("Summary:")
print(f"Total number of poems stored: {num_poem}")
print(f"Total number of authors: {num_author}")
#print(f"Top 20 Authors are: {top20}")

print("The top 20 Authors are:")
for i in range(0,20):
    print(f" Number {i+1} is {top20[i]} has {(poem_data['Author'].value_counts()[i])} poems")
          

Summary:
Total number of poems stored: 2040
Total number of authors: 2040
The top 20 Authors are:
 Number 1 is William Shakespeare has 52 poems
 Number 2 is John Donne has 28 poems
 Number 3 is Edmund Spenser has 23 poems
 Number 4 is Sir  Thomas Wyatt has 19 poems
 Number 5 is Robert Browning has 19 poems
 Number 6 is Anonymous has 19 poems
 Number 7 is John Keats has 17 poems
 Number 8 is Algernon Charles Swinburne has 15 poems
 Number 9 is Thomas Campion has 15 poems
 Number 10 is Robert Burns has 13 poems
 Number 11 is George Meredith has 13 poems
 Number 12 is Alfred, Lord Tennyson has 13 poems
 Number 13 is William Butler Yeats has 13 poems
 Number 14 is Aphra Behn has 12 poems
 Number 15 is Andrew Marvell has 12 poems
 Number 16 is Christina Rossetti has 12 poems
 Number 17 is Brenda Shaughnessy has 12 poems
 Number 18 is Robert Herrick has 11 poems
 Number 19 is Amy Lowell has 11 poems
 Number 20 is Walt Whitman has 11 poems


In [33]:
# Part 2_1. 
# Read data from CleanOutputLoveOutput.xlsx, collect information about words,sentences, paragraphs, 
# and punctuations of poems in this data set.

In [34]:
# read excel file CleanOutputLoveOutput.xlsx into df
file_path = "C:\\Users\\Yuliya.DESKTOP-FMS37R3\\Documents\\Fordham Fall 2021 Classes\\CISC6210 NLP\\Homework\\CleanOutputLoveOutput.xlsx"
CleanOut_df = pd.read_excel(file_path)

In [35]:
#create new table
column_names = ["PoemID", "Author", "LengthOne", "LengthTwo", "NumLine", "NumPara", "NumSent", "NumComma"]
stat_df = pd.DataFrame(columns = column_names)

In [36]:
#add author names to new df
stat_df["Author"] = CleanOut_df["Author"].copy()

#populate col PoemID as row index of each poem from CleanOutputLoveOutput.xlsx file 
stat_df["PoemID"]= CleanOut_df["Author"].index

In [37]:
#LengthOne: the total number of tokens in this poem, punctuation marks are not included.
for i in range(0, len(CleanOut_df)):
    num_token = 0
    num_token = CleanOut_df["Body"][i].split()
    stat_df["LengthOne"][i] = int(len(num_token))

C:\Users\Yuliya.DESKTOP-FMS37R3\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [38]:
# LengthTwo: the total number of tokens in this poem, punctuation marks are included.
for i in range(0, len(CleanOut_df)):    
    num_tokenize = 0  
    num_tokenize = nltk.word_tokenize(CleanOut_df["Body"][i])
    stat_df["LengthTwo"][i] = int(len(num_tokenize))

C:\Users\Yuliya.DESKTOP-FMS37R3\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [39]:
for i in range(0, len(CleanOut_df)):
    
    #NumLine: line breaks plus paragraph breaks
    stat_df["NumLine"][i] = CleanOut_df["Body"][i].count("[L]") + CleanOut_df["Body"][i].count("[P]")
    #NumPara: paragraph breaks
    stat_df["NumPara"][i] = CleanOut_df["Body"][i].count("[P]")
    #NumComma: commas
    stat_df["NumComma"][i] = CleanOut_df["Body"][i].count(",")

C:\Users\Yuliya.DESKTOP-FMS37R3\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\Yuliya.DESKTOP-FMS37R3\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Yuliya.DESKTOP-FMS37R3\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/

In [40]:
# NumSent: the total number of complete sentences (use nltk)
# Remove [L] and [P] and replace with spaces before counting sentenses to get accurate results. 
# Otherwise line end [L] at the end are counted as a separate sentense. 

#sent_list = []

CleanOut_df['Body'] = CleanOut_df['Body'].str.replace('[L]', ' ', regex=False)
CleanOut_df['Body'] = CleanOut_df['Body'].str.replace('[P]', ' ', regex=False)

for i in range(0, len(CleanOut_df)):

    stat_df['NumSent'][i] = int(len(nltk.sent_tokenize(CleanOut_df['Body'][i])))
    #sent_list.append(int(len(nltk.sent_tokenize(CleanOut_df['Body'][i]))))


C:\Users\Yuliya.DESKTOP-FMS37R3\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [72]:
#print(stat_df[0:5])

In [42]:
stat_df.describe()

,PoemID
count,2040.000000
mean,1019.500000
std,589.041594
min,0.000000
25%,509.750000
50%,1019.500000
75%,1529.250000
max,2039.000000


In [43]:
stat_df.count()

PoemID       2040
Author       2040
LengthOne    2040
LengthTwo    2040
NumLine      2040
NumPara      2040
NumSent      2040
NumComma     2040
dtype: int64

In [44]:
stat_df.min()

PoemID                   0
Author       A. E. Housman
LengthOne               13
LengthTwo               21
NumLine                  0
NumPara                  0
NumSent                  1
NumComma                 0
dtype: object

In [45]:
stat_df.max()

PoemID          2039
Author       sam sax
LengthOne       5827
LengthTwo       9247
NumLine          971
NumPara          147
NumSent          427
NumComma         663
dtype: object

In [46]:
stat_df.mean()

PoemID       1019.500000
LengthOne     245.232843
LengthTwo     421.606373
NumLine        42.267157
NumPara         0.424510
NumSent        12.539216
NumComma       20.706373
dtype: float64

In [47]:
stat_df.std()

PoemID       589.041594
LengthOne    413.503223
LengthTwo    675.345344
NumLine       63.952500
NumPara        3.613118
NumSent       22.854068
NumComma      40.448037
dtype: float64

In [49]:
# Part 2_2. 

In [50]:
#create 3 new tables with 5 columns each
column_names = ["PoemID", "Author", "Body", "Length", "UniCount"]

table1_df = pd.DataFrame(columns = column_names)

table2_df = pd.DataFrame(columns = column_names)

table3_df = pd.DataFrame(columns = column_names)

#add Author names to new df
table1_df["Author"] = CleanOut_df["Author"].copy()
table2_df["Author"] = CleanOut_df["Author"].copy()
table3_df["Author"] = CleanOut_df["Author"].copy()

#populate col PoemID as row index of each poem from CleanOutputLoveOutput.xlsx file 
table1_df["PoemID"]= CleanOut_df["Author"].index
table2_df["PoemID"]= CleanOut_df["Author"].index
table3_df["PoemID"]= CleanOut_df["Author"].index

In [51]:
# The total number of tokens in this poem, punctuation marks are included.
table1_df["Body"]= stat_df["LengthTwo"].copy() 

In [52]:
# populate the table 1 column "Length"
for i in range(0, len(CleanOut_df)):
    table1_df['Length'][i] = int(len(CleanOut_df['Body'][i]))

C:\Users\Yuliya.DESKTOP-FMS37R3\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [53]:
# populate the table 1 column "UniCount" with unique values of words in each poem vocabulary_size = len(set(text))
for i in range(0, len(CleanOut_df)):
    table1_df['UniCount'][i] = len(set(CleanOut_df['Body'][i]))

C:\Users\Yuliya.DESKTOP-FMS37R3\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [55]:
# The second table is for tokenized words after stop words removal. (use nltk English stopwords list). 
# get stopwords 
#print(stopwords.words('english'))

In [56]:
from gensim.parsing.preprocessing import remove_stopwords

# remove stopwords from Body column of every poem in df
stopwords = nltk.corpus.stopwords.words('english')

for i in range(0, len(CleanOut_df)):
     CleanOut_df["Body"][i] = remove_stopwords(CleanOut_df["Body"][i].lower())  #filtered "Body" column of poems with stopwords removed

In [76]:
print(CleanOut_df["Body"][1])

['i', "'m", 'emperor', "'s", 'bitter', 'ca', "n't", 'sweet', 'place', 'face', 'face', 'look', 'touch', 'oak', 'blossom', 'strip', 'swell', 'avenu', 'pavement', 'river', 'shirt', 'ca', "n't", 'night', 'gather', 'new', 'day', 'sweet', 'place', 'tomorrow', 'whisper', 'tonight', "'s", 'light', 'kiss', 'author', 'want', 'stop', 'you', "'ll", 'ruin', 'citi', 'woman', 'interrupt', 'sleep']


In [57]:
# populate the table 2 column "Length"
for i in range(0, len(CleanOut_df)):
    table2_df['Length'][i] = int(len(CleanOut_df['Body'][i]))


C:\Users\Yuliya.DESKTOP-FMS37R3\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [77]:
# populate the table 2 column "Body" - Tokenized words in poem the total number of tokens in this poem, punctuation marks are included.
for i in range(0, len(CleanOut_df)):  
    table2_df['Body'][i] = len(CleanOut_df["Body"][i])

C:\Users\Yuliya.DESKTOP-FMS37R3\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [81]:
table2_df[0:3]

,PoemID,Author,Body,Length,UniCount
0,0,Roddy Lumsden,177,899,34
1,1,Suzanne Gardinier,45,272,25
2,2,Jeff Daniel Marion,60,338,30


In [58]:
# populate the table 2 column "UniCount" with the vocabulary size - number of unique words used in each poem 
for i in range(0, len(CleanOut_df)):
    table2_df['UniCount'][i] = len(set(CleanOut_df['Body'][i]))

C:\Users\Yuliya.DESKTOP-FMS37R3\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [60]:
# The third table is for tokenized words after stop words removal and stemming. (use one stemmer - porter’s from nltk)

from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
#porter = nltk.PorterStemmer()

ps = PorterStemmer()

for i in range(0, len(CleanOut_df)):
     CleanOut_df["Body"][i] = word_tokenize(CleanOut_df["Body"][i])
        
#function for stemming 
def stemming(text):
    porter = PorterStemmer()
    
    result=[]
    for word in text:
        result.append(porter.stem(word))
    return result        


In [61]:
# appluy stemming to the poems 
CleanOut_df['Body']=CleanOut_df['Body'].apply(stemming)
#CleanOut_df.head()

In [62]:
# populae Body of table 3 
for i in range(0, len(CleanOut_df)):
    table3_df["Body"][i] = int(len(CleanOut_df["Body"][i]))

C:\Users\Yuliya.DESKTOP-FMS37R3\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [82]:
# populae Length of table 3 
for i in range(0, len(CleanOut_df)):
    table3_df['Length'][i] = sum([len(element) for element in CleanOut_df['Body'][i]])


C:\Users\Yuliya.DESKTOP-FMS37R3\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [83]:
print(table3_df['Length'])

0       687
1       205
2       255
3        51
4       809
       ... 
2035    478
2036    574
2037    329
2038    511
2039    441
Name: Length, Length: 2040, dtype: object


In [84]:
print(table3_df[0:3])

   PoemID              Author Body Length UniCount
0       0       Roddy Lumsden  177    687      131
1       1   Suzanne Gardinier   45    205       39
2       2  Jeff Daniel Marion   60    255       48


In [64]:
# populate the table 3 column "UniCount" with the vocabulary size - number of unique words used in each poem 
for i in range(0, len(CleanOut_df)):
    table3_df['UniCount'][i] = len(set(CleanOut_df['Body'][i]))


C:\Users\Yuliya.DESKTOP-FMS37R3\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [85]:
############# SAVE all 4 tables to the same excel file ################

from openpyxl import load_workbook

path = "C:\\Users\\Yuliya.DESKTOP-FMS37R3\\Documents\\Fordham Fall 2021 Classes\\CISC6210 NLP\\Homework\\ProcessedLoveOutput.xlsx"

with pd.ExcelWriter(path, engine='xlsxwriter') as writer:    
    stat_df.to_excel(writer, 'Statistics')   
    table1_df.to_excel(writer, 'Table1')  
    table2_df.to_excel(writer, 'Table2') 
    table3_df.to_excel(writer, 'Table3') 
    
    writer.save()    